In [1]:
import pandas as pd
import numpy as np
import joblib
import json
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score, 
                           precision_score, recall_score, f1_score, roc_auc_score)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

class StuntingModelTester:
    """
    Kelas untuk menguji model Decision Tree prediksi stunting
    """
    
    def __init__(self, model_path='stunting_decision_tree_model.pkl',
                 scaler_path='stunting_scaler.pkl',
                 selector_path='stunting_feature_selector.pkl',
                 metadata_path='stunting_decision_tree_metadata.json'):
        """
        Inisialisasi tester dengan memuat model dan komponen preprocessing
        """
        self.model_path = model_path
        self.scaler_path = scaler_path
        self.selector_path = selector_path
        self.metadata_path = metadata_path
        
        self.model = None
        self.scaler = None
        self.selector = None
        self.metadata = None
        self.feature_names = None
        
        self._load_model_components()
    
    def _load_model_components(self):
        """
        Memuat semua komponen model yang diperlukan
        """
        try:
            print("Loading model components...")
            
            # Load model
            self.model = joblib.load(self.model_path)
            print(f"✅ Model loaded from {self.model_path}")
            
            # Load scaler
            self.scaler = joblib.load(self.scaler_path)
            print(f"✅ Scaler loaded from {self.scaler_path}")
            
            # Load feature selector
            self.selector = joblib.load(self.selector_path)
            print(f"✅ Feature selector loaded from {self.selector_path}")
            
            # Load metadata
            with open(self.metadata_path, 'r') as f:
                self.metadata = json.load(f)
            print(f"✅ Metadata loaded from {self.metadata_path}")
            
            # Get feature names
            self.feature_names = self.metadata['features_used']
            print(f"✅ Feature names loaded: {self.feature_names}")
            
        except FileNotFoundError as e:
            print(f"❌ Error: File not found - {e}")
            raise
        except Exception as e:
            print(f"❌ Error loading model components: {e}")
            raise
    
    def preprocess_data(self, X):
        """
        Preprocessing data untuk testing (scaling dan feature selection)
        """
        try:
            # Scale features
            X_scaled = self.scaler.transform(X)
            
            # Select features
            X_selected = self.selector.transform(X_scaled)
            
            return X_selected
            
        except Exception as e:
            print(f"❌ Error in preprocessing: {e}")
            raise
    
    def predict_single(self, data_dict):
        """
        Prediksi untuk satu sampel data
        
        Args:
            data_dict: Dictionary berisi data sampel
            
        Returns:
            Dictionary berisi hasil prediksi dan probabilitas
        """
        try:
            # Convert to DataFrame
            df = pd.DataFrame([data_dict])
            
            # Ensure all required features are present
            for feature in self.feature_names:
                if feature not in df.columns:
                    print(f"⚠️  Warning: Feature '{feature}' not found, setting to 0")
                    df[feature] = 0
            
            # Select only required features
            X = df[self.feature_names]
            
            # Preprocess
            X_processed = self.preprocess_data(X)
            
            # Predict
            prediction = self.model.predict(X_processed)[0]
            probabilities = self.model.predict_proba(X_processed)[0]
            
            result = {
                'prediction': int(prediction),
                'prediction_label': 'Stunting' if prediction == 1 else 'Normal',
                'probability_normal': float(probabilities[0]),
                'probability_stunting': float(probabilities[1]),
                'confidence': float(max(probabilities)),
                'input_data': data_dict
            }
            
            return result
            
        except Exception as e:
            print(f"❌ Error in single prediction: {e}")
            return None
    
    def predict_batch(self, X, y=None):
        """
        Prediksi untuk batch data
        
        Args:
            X: DataFrame berisi fitur
            y: Array berisi label sebenarnya (optional)
            
        Returns:
            Dictionary berisi prediksi dan evaluasi (jika y tersedia)
        """
        try:
            # Ensure all required features are present
            missing_features = []
            for feature in self.feature_names:
                if feature not in X.columns:
                    missing_features.append(feature)
                    X[feature] = 0
            
            if missing_features:
                print(f"⚠️  Warning: Missing features set to 0: {missing_features}")
            
            # Select only required features
            X_selected = X[self.feature_names]
            
            # Preprocess
            X_processed = self.preprocess_data(X_selected)
            
            # Predict
            predictions = self.model.predict(X_processed)
            probabilities = self.model.predict_proba(X_processed)
            
            result = {
                'predictions': predictions,
                'probabilities': probabilities,
                'predictions_labels': ['Stunting' if p == 1 else 'Normal' for p in predictions]
            }
            
            # Evaluate if true labels provided
            if y is not None:
                evaluation = self.evaluate_predictions(y, predictions, probabilities[:, 1])
                result['evaluation'] = evaluation
            
            return result
            
        except Exception as e:
            print(f"❌ Error in batch prediction: {e}")
            return None
    
    def evaluate_predictions(self, y_true, y_pred, y_proba):
        """
        Evaluasi performa prediksi
        """
        try:
            evaluation = {
                'accuracy': accuracy_score(y_true, y_pred),
                'precision': precision_score(y_true, y_pred, zero_division=0),
                'recall': recall_score(y_true, y_pred, zero_division=0),
                'f1_score': f1_score(y_true, y_pred, zero_division=0),
                'confusion_matrix': confusion_matrix(y_true, y_pred).tolist(),
                'classification_report': classification_report(y_true, y_pred, 
                                                              target_names=['Normal', 'Stunting'],
                                                              output_dict=True)
            }
            
            # ROC AUC if possible
            try:
                evaluation['roc_auc'] = roc_auc_score(y_true, y_proba)
            except:
                evaluation['roc_auc'] = None
            
            return evaluation
            
        except Exception as e:
            print(f"❌ Error in evaluation: {e}")
            return None
    
    def generate_test_report(self, X_test, y_test, save_report=True):
        """
        Generate comprehensive test report
        """
        print("="*60)
        print("LAPORAN PENGUJIAN MODEL STUNTING DECISION TREE")
        print("="*60)
        
        # Model info
        print(f"\nINFORMASI MODEL:")
        print(f"- Tipe Model: {self.metadata['model_type']}")
        print(f"- Tanggal Training: {self.metadata['training_date']}")
        print(f"- Jumlah Fitur: {len(self.feature_names)}")
        print(f"- Fitur yang Digunakan: {', '.join(self.feature_names)}")
        
        # Model parameters
        print(f"\nPARAMETER MODEL:")
        for key, value in self.metadata['parameters'].items():
            print(f"- {key}: {value}")
        
        # Predict on test data
        print(f"\nTESTING PADA DATA UJI...")
        print(f"Jumlah sampel test: {len(X_test)}")
        
        result = self.predict_batch(X_test, y_test)
        
        if result and 'evaluation' in result:
            eval_data = result['evaluation']
            
            print(f"\nHASIL EVALUASI:")
            print(f"- Accuracy: {eval_data['accuracy']:.3f}")
            print(f"- Precision: {eval_data['precision']:.3f}")
            print(f"- Recall: {eval_data['recall']:.3f}")
            print(f"- F1-Score: {eval_data['f1_score']:.3f}")
            if eval_data['roc_auc']:
                print(f"- ROC AUC: {eval_data['roc_auc']:.3f}")
            
            # Confusion Matrix
            cm = np.array(eval_data['confusion_matrix'])
            print(f"\nCONFUSION MATRIX:")
            print("             Predicted")
            print("Actual    Normal  Stunting")
            print(f"Normal      {cm[0,0]:6d}  {cm[0,1]:8d}")
            print(f"Stunting    {cm[1,0]:6d}  {cm[1,1]:8d}")
            
            # Performance comparison with training
            print(f"\nPERBANDINGAN DENGAN PERFORMA TRAINING:")
            train_f1 = self.metadata['performance']['cv_f1_mean']
            test_f1 = eval_data['f1_score']
            
            print(f"- Training F1 (CV): {train_f1:.3f}")
            print(f"- Testing F1: {test_f1:.3f}")
            print(f"- Selisih: {abs(train_f1 - test_f1):.3f}")
            
            if abs(train_f1 - test_f1) > 0.1:
                print("⚠️  Warning: Perbedaan performa signifikan, kemungkinan overfitting!")
            else:
                print("✅ Performa konsisten antara training dan testing")
            
            # Save report if requested
            if save_report:
                report_data = {
                    'test_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'model_info': self.metadata,
                    'test_results': eval_data,
                    'test_data_size': len(X_test),
                    'performance_comparison': {
                        'training_f1': train_f1,
                        'testing_f1': test_f1,
                        'difference': abs(train_f1 - test_f1)
                    }
                }
                
                with open('stunting_test_report.json', 'w') as f:
                    json.dump(report_data, f, indent=2)
                print(f"\n📄 Laporan tersimpan: stunting_test_report.json")
            
            # Visualization
            self._plot_test_results(eval_data, cm)
            
            return result
        else:
            print("❌ Gagal melakukan evaluasi")
            return None
    
    def _plot_test_results(self, eval_data, cm):
        """
        Plot test results
        """
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        
        # Confusion Matrix
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                   xticklabels=['Normal', 'Stunting'],
                   yticklabels=['Normal', 'Stunting'], ax=axes[0,0])
        axes[0,0].set_title('Confusion Matrix')
        axes[0,0].set_xlabel('Predicted')
        axes[0,0].set_ylabel('Actual')
        
        # Performance Metrics
        metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
        values = [eval_data['accuracy'], eval_data['precision'], 
                 eval_data['recall'], eval_data['f1_score']]
        
        bars = axes[0,1].bar(metrics, values, color=['skyblue', 'lightgreen', 'lightcoral', 'gold'])
        axes[0,1].set_title('Performance Metrics')
        axes[0,1].set_ylabel('Score')
        axes[0,1].set_ylim(0, 1)
        
        # Add value labels on bars
        for bar, value in zip(bars, values):
            axes[0,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                          f'{value:.3f}', ha='center', va='bottom')
        
        # Performance comparison
        train_metrics = ['Training F1', 'Testing F1']
        train_f1 = self.metadata['performance']['cv_f1_mean']
        test_f1 = eval_data['f1_score']
        comparison_values = [train_f1, test_f1]
        
        bars = axes[1,0].bar(train_metrics, comparison_values, 
                            color=['lightblue', 'orange'])
        axes[1,0].set_title('Training vs Testing F1-Score')
        axes[1,0].set_ylabel('F1-Score')
        axes[1,0].set_ylim(0, 1)
        
        for bar, value in zip(bars, comparison_values):
            axes[1,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                          f'{value:.3f}', ha='center', va='bottom')
        
        # Feature importance (from metadata)
        if 'feature_importance' in self.metadata:
            features = list(self.metadata['feature_importance'].keys())
            importance = list(self.metadata['feature_importance'].values())
            
            axes[1,1].barh(features, importance)
            axes[1,1].set_title('Feature Importance')
            axes[1,1].set_xlabel('Importance')
        else:
            axes[1,1].text(0.5, 0.5, 'Feature Importance\nNot Available', 
                          ha='center', va='center', transform=axes[1,1].transAxes)
            axes[1,1].set_title('Feature Importance')
        
        plt.tight_layout()
        plt.savefig('stunting_test_results_visualization.png', dpi=300, bbox_inches='tight')
        print("📊 Visualisasi tersimpan: stunting_test_results_visualization.png")
        plt.show()
    
    def test_new_samples(self, samples_data):
        """
        Test dengan data sampel baru
        
        Args:
            samples_data: List of dictionaries berisi data sampel
        """
        print("="*50)
        print("TESTING DENGAN SAMPEL BARU")
        print("="*50)
        
        for i, sample in enumerate(samples_data, 1):
            print(f"\nSAMPEL {i}:")
            print(f"Data input: {sample}")
            
            result = self.predict_single(sample)
            
            if result:
                print(f"Prediksi: {result['prediction_label']}")
                print(f"Confidence: {result['confidence']:.3f}")
                print(f"Probabilitas Normal: {result['probability_normal']:.3f}")
                print(f"Probabilitas Stunting: {result['probability_stunting']:.3f}")
                
                # Interpretasi
                if result['prediction'] == 1:
                    if result['confidence'] > 0.8:
                        print("🔴 RISIKO TINGGI - Perlu perhatian medis segera")
                    elif result['confidence'] > 0.6:
                        print("🟡 RISIKO SEDANG - Monitoring dan intervensi diperlukan")
                    else:
                        print("🟢 RISIKO RENDAH - Tetap monitor perkembangan")
                else:
                    print("✅ NORMAL - Pertahankan pola pertumbuhan")
            else:
                print("❌ Gagal melakukan prediksi")
        
        return True

# Fungsi utility untuk membuat data test
def create_sample_test_data():
    """
    Membuat data sampel untuk testing
    """
    samples = [
        {
            'Umur (bulan)': 24,
            'Jenis Kelamin Encoded': 0,  # Laki-laki
            'Age_Groups': 1,
            'Height_Gender_Interaction': 85.0,
            'Height_Age_Ratio': 3.5
        },
        {
            'Umur (bulan)': 36,
            'Jenis Kelamin Encoded': 1,  # Perempuan
            'Age_Groups': 2,
            'Height_Gender_Interaction': 95.0,
            'Height_Age_Ratio': 2.6
        },
        {
            'Umur (bulan)': 12,
            'Jenis Kelamin Encoded': 0,  # Laki-laki
            'Age_Groups': 0,
            'Height_Gender_Interaction': 70.0,
            'Height_Age_Ratio': 5.8
        }
    ]
    
    return samples

# Fungsi utama untuk testing
def run_model_testing():
    """
    Fungsi utama untuk menjalankan testing model
    """
    try:
        # Initialize tester
        tester = StuntingModelTester()
        
        # Test dengan sampel baru
        print("Testing dengan sampel contoh...")
        sample_data = create_sample_test_data()
        tester.test_new_samples(sample_data)
        
        # Jika ada data test tersedia, lakukan evaluasi lengkap
        try:
            # Coba load data test (sesuaikan dengan format data Anda)
            # test_data = pd.read_csv('test_data.csv')  # Ganti dengan path file test
            # X_test = test_data[tester.feature_names]
            # y_test = test_data['Is_Stunting']
            # tester.generate_test_report(X_test, y_test)
            print("\n💡 Untuk evaluasi lengkap, siapkan data test dan uncomment kode di atas")
            
        except:
            print("\n💡 Data test tidak tersedia untuk evaluasi lengkap")
        
        print("\n✅ Testing selesai!")
        
    except Exception as e:
        print(f"❌ Error dalam testing: {e}")

if __name__ == "__main__":
    run_model_testing()

Loading model components...
✅ Model loaded from stunting_decision_tree_model.pkl
✅ Scaler loaded from stunting_scaler.pkl
✅ Feature selector loaded from stunting_feature_selector.pkl
✅ Metadata loaded from stunting_decision_tree_metadata.json
✅ Feature names loaded: ['Umur (bulan)', 'Jenis Kelamin Encoded', 'Age_Groups', 'Height_Gender_Interaction', 'Height_Age_Ratio']
Testing dengan sampel contoh...
TESTING DENGAN SAMPEL BARU

SAMPEL 1:
Data input: {'Umur (bulan)': 24, 'Jenis Kelamin Encoded': 0, 'Age_Groups': 1, 'Height_Gender_Interaction': 85.0, 'Height_Age_Ratio': 3.5}
Prediksi: Normal
Confidence: 1.000
Probabilitas Normal: 1.000
Probabilitas Stunting: 0.000
✅ NORMAL - Pertahankan pola pertumbuhan

SAMPEL 2:
Data input: {'Umur (bulan)': 36, 'Jenis Kelamin Encoded': 1, 'Age_Groups': 2, 'Height_Gender_Interaction': 95.0, 'Height_Age_Ratio': 2.6}
Prediksi: Normal
Confidence: 0.970
Probabilitas Normal: 0.970
Probabilitas Stunting: 0.030
✅ NORMAL - Pertahankan pola pertumbuhan

SAMPEL 3